# Geocoding
## 使用データのロード

In [1]:
import os
print(os.getcwd())

c:\Users\hkato\Desktop\MHLW\RDD\data\tukuba-geocode\code


In [2]:
import pandas as pd
post_list = pd.read_csv("../raw/MLIT/08220_2019.csv", encoding="SHIFT-JIS")
post_list

,都道府県コード,都道府県名,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,緯度,経度,原典資料コード,大字・字・丁目区分コード
0,8,茨城県,8220,つくば市,82200001000,赤塚,36.042677,140.123150,0,1
1,8,茨城県,8220,つくば市,82200002000,明石,36.182065,140.053737,0,1
2,8,茨城県,8220,つくば市,82200003000,旭,36.112890,140.072495,0,1
3,8,茨城県,8220,つくば市,82200004000,安食,36.167751,140.016345,0,1
4,8,茨城県,8220,つくば市,82200005000,あしび野,35.960604,140.112185,0,1
...,...,...,...,...,...,...,...,...,...,...
297,8,茨城県,8220,つくば市,82200237000,上河原崎元宮本,36.075704,140.036447,0,1
298,8,茨城県,8220,つくば市,82200238000,上河原崎,36.079024,140.039170,0,1
299,8,茨城県,8220,つくば市,82200999999,春風台,36.116077,140.124268,0,1
300,8,茨城県,8220,つくば市,82200999999,流星台,36.104861,140.122227,0,1


In [3]:
admin_list = pd.read_csv("../raw/tukuba-admin.csv", encoding="SHIFT-JIS")
admin_list

,name,address,zip,latitude,longitude
0,大穂保健センター,茨城県つくば市筑穂1-10-4,3003257,36.137015,140.082554
1,桜保健センター,茨城県つくば市流星台61-1,3050008,36.095188,140.128661
2,谷田部保健センター,茨城県つくば市谷田部4774-18,3050861,36.037118,140.074604
3,健康増進課,茨城県つくば市研究学園1-1-1,3058555,36.083653,140.076709


In [4]:
admin_list["label"] = ["oho", "sakura", "yatabe", "hall"]
admin_list

,name,address,zip,latitude,longitude,label
0,大穂保健センター,茨城県つくば市筑穂1-10-4,3003257,36.137015,140.082554,oho
1,桜保健センター,茨城県つくば市流星台61-1,3050008,36.095188,140.128661,sakura
2,谷田部保健センター,茨城県つくば市谷田部4774-18,3050861,36.037118,140.074604,yatabe
3,健康増進課,茨城県つくば市研究学園1-1-1,3058555,36.083653,140.076709,hall


## `GeoPy`による直線距離の計算

`geopy.distance`を用いる。計算方法は二種類

1. 測地距離（`import geodesic`）：楕円である地球上で、2ポイント間の最短距離を結んだ距離
1. 大円距離（`import great_circle`）：球面上の2点間の長さが最短となる距離（早いが精度悪い）

測地距離を使用する

In [5]:
from geopy.distance import geodesic

for i in range(len(admin_list)):
    to = ((admin_list.at[i, "latitude"]), (admin_list.at[i, "longitude"]))
    label = "dist_" + admin_list.at[i, "label"]
    for j in range(len(post_list)):
        start = ((post_list.at[j, "緯度"]), (post_list.at[j, "経度"]))
        x = geodesic(to, start).km
        post_list.at[j, label] = x

post_list

,都道府県コード,都道府県名,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,緯度,経度,原典資料コード,大字・字・丁目区分コード,dist_oho,dist_sakura,dist_yatabe,dist_hall
0,8,茨城県,8220,つくば市,82200001000,赤塚,36.042677,140.123150,0,1,11.087947,5.847745,4.418146,6.178854
1,8,茨城県,8220,つくば市,82200002000,明石,36.182065,140.053737,0,1,5.631338,11.764594,16.192805,11.113957
2,8,茨城県,8220,つくば市,82200003000,旭,36.112890,140.072495,0,1,2.825992,5.425518,8.409804,3.266250
3,8,茨城県,8220,つくば市,82200004000,安食,36.167751,140.016345,0,1,6.865235,12.924460,15.415237,10.798468
4,8,茨城県,8220,つくば市,82200005000,あしび野,35.960604,140.112185,0,1,19.755827,15.007029,9.141176,14.022946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,8,茨城県,8220,つくば市,82200237000,上河原崎元宮本,36.075704,140.036447,0,1,7.969882,8.582151,5.490930,3.732243
298,8,茨城県,8220,つくば市,82200238000,上河原崎,36.079024,140.039170,0,1,7.527678,8.257104,5.640333,3.419971
299,8,茨城県,8220,つくば市,82200999999,春風台,36.116077,140.124268,0,1,4.415686,2.351327,9.837272,5.593284
300,8,茨城県,8220,つくば市,82200999999,流星台,36.104861,140.122227,0,1,5.048340,1.219715,8.654809,4.726620


## Google Maps Platform (Distance Matrix API)による運転時間の推定

[この記事](https://leben.mobi/blog/python_distance_matrix_api/python/)を参考にして構築。APIの仕様の詳細は[ここ](https://developers.google.com/maps/documentation/javascript/distancematrix?hl=ja)

In [6]:
import json
import requests

apikey = open("../raw/distance-matrix-api.txt").read()
api = 'https://maps.googleapis.com/maps/api/distancematrix/json'

for i in range(len(admin_list)):
    goal = format(admin_list.at[i, "latitude"]) + "," + format(admin_list.at[i, "longitude"])
    drive_dist_label = 'drive_dist_' + admin_list.at[i, 'label']
    drive_time_label = 'drive_time_' + admin_list.at[i, 'label']
    for j in range(len(post_list)):
        origin = format(post_list.at[j, "緯度"]) + ',' + format(post_list.at[j, "経度"])
        params = {
            'key': apikey,
            'origins': origin,
            'destinations': goal,
            'mode': 'driving',
            # 'departure_time': 'now',
            'trafficModel': 'bestguess',
            'avoidHighways': True,
            'avoidTolls': True
        }
        response = requests.get(api, params)
        parse_response = json.loads(response.text)

        if parse_response['status'] != 'OK':
            continue
        
        elements = parse_response['rows'][0]['elements'][0]

        if elements['status'] == 'OK':
            drive_dist = elements['distance']['value'] / 1000
            drive_time = elements['duration']['value'] / 60
        else:
            drive_dist = None
            drive_time = None
        
        post_list.at[j, drive_dist_label] = drive_dist
        post_list.at[j, drive_time_label] = drive_time

post_list

,都道府県コード,都道府県名,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,緯度,経度,原典資料コード,大字・字・丁目区分コード,...,dist_yatabe,dist_hall,drive_dist_oho,drive_time_oho,drive_dist_sakura,drive_time_sakura,drive_dist_yatabe,drive_time_yatabe,drive_dist_hall,drive_time_hall
0,8,茨城県,8220,つくば市,82200001000,赤塚,36.042677,140.123150,0,1,...,4.418146,6.178854,12.430,21.883333,7.502,15.516667,4.820,10.266667,8.073,17.116667
1,8,茨城県,8220,つくば市,82200002000,明石,36.182065,140.053737,0,1,...,16.192805,11.113957,7.153,11.700000,14.151,24.000000,18.981,28.766667,14.317,23.450000
2,8,茨城県,8220,つくば市,82200003000,旭,36.112890,140.072495,0,1,...,8.409804,3.266250,4.206,7.950000,7.632,15.200000,9.775,17.350000,5.085,12.416667
3,8,茨城県,8220,つくば市,82200004000,安食,36.167751,140.016345,0,1,...,15.415237,10.798468,8.770,15.733333,17.174,29.283333,18.513,28.733333,14.272,24.550000
4,8,茨城県,8220,つくば市,82200005000,あしび野,35.960604,140.112185,0,1,...,9.141176,14.022946,23.075,37.733333,25.402,33.616667,11.672,18.683333,16.898,28.483333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,8,茨城県,8220,つくば市,82200237000,上河原崎元宮本,36.075704,140.036447,0,1,...,5.490930,3.732243,11.127,17.566667,10.360,20.816667,6.813,12.583333,4.742,9.366667
298,8,茨城県,8220,つくば市,82200238000,上河原崎,36.079024,140.039170,0,1,...,5.640333,3.419971,10.085,15.216667,10.400,20.050000,6.853,11.616667,4.782,8.583333
299,8,茨城県,8220,つくば市,82200999999,春風台,36.116077,140.124268,0,1,...,9.837272,5.593284,5.497,11.566667,3.405,8.000000,12.075,25.166667,7.008,17.566667
300,8,茨城県,8220,つくば市,82200999999,流星台,36.104861,140.122227,0,1,...,8.654809,4.726620,5.796,11.566667,1.804,4.133333,10.474,21.316667,7.098,16.300000


## データ確認

- 欠損値確認
- 記述統計

In [7]:
post_list.isnull().any()

都道府県コード              False
都道府県名                False
市区町村コード              False
市区町村名                False
大字町丁目コード             False
大字町丁目名               False
緯度                   False
経度                   False
原典資料コード              False
大字・字・丁目区分コード         False
dist_oho             False
dist_sakura          False
dist_yatabe          False
dist_hall            False
drive_dist_oho       False
drive_time_oho       False
drive_dist_sakura    False
drive_time_sakura    False
drive_dist_yatabe    False
drive_time_yatabe    False
drive_dist_hall      False
drive_time_hall      False
dtype: bool

In [8]:
post_list.describe().loc[:, 'dist_oho':'drive_time_hall']

,dist_oho,dist_sakura,dist_yatabe,dist_hall,drive_dist_oho,drive_time_oho,drive_dist_sakura,drive_time_sakura,drive_dist_yatabe,drive_time_yatabe,drive_dist_hall,drive_time_hall
count,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000
mean,8.401957,7.119571,7.506658,6.305868,10.197669,18.201049,9.700864,18.551656,9.319401,16.869150,8.239288,16.236810
std,4.486445,3.588999,4.259739,3.454903,5.040957,8.217285,5.128858,6.911186,5.395813,8.096513,4.142997,6.275108
min,0.104616,0.399518,0.434891,0.040646,0.307000,1.466667,0.736000,2.916667,0.526000,1.316667,0.132000,0.800000
25%,5.120009,4.252366,4.595608,3.625097,6.687500,12.087500,5.667750,13.137500,5.663750,11.287500,5.029500,11.662500
50%,7.461906,7.024308,6.468411,5.847202,9.036000,16.650000,9.131000,18.975000,7.983000,15.033333,7.838500,16.616667
75%,11.601651,9.797007,9.726946,8.428460,14.246500,24.550000,12.766000,23.883333,11.851750,21.708333,10.769000,20.004167
max,20.930879,16.092327,20.805902,15.643619,23.935000,38.483333,26.263000,34.350000,27.666000,42.800000,19.861000,34.900000


## データの書き出し

In [9]:
post_list.to_csv('../tukuba-geocode.csv', encoding='SHIFT-JIS')